In [1]:
import cv2
import numpy as np
import imutils
from skimage import data
from skimage.filters import threshold_otsu
import os
     
def order_points(pts):
    # initialize a list of coordinates that will be ordered
    #  such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
     
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
     
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
     
    # return the ordered coordinates
    return rect
     
def four_point_transform(image, pts):
        # obtain a consistent order of the points and unpack them
        # individually
        rect = order_points(pts)
        (tl, tr, br, bl) = rect
     
        # compute the width of the new image, which will be the
        # maximum distance between bottom-right and bottom-left
        # x-coordinates or the top-right and top-left x-coordinates
        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))
     
        # compute the height of the new image, which will be the
        # maximum distance between the top-right and bottom-right
        # y-coordinates or the top-left and bottom-left y-coordinates
        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))
     
        # now that we have the dimensions of the new image, construct
        # the set of destination points to obtain a "birds eye view",
        # (i.e. top-down view) of the image, again specifying points
        # in the top-left, top-right, bottom-right, and bottom-left
        # order
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]], dtype = "float32")
     
        # compute the perspective transform matrix and then apply it
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
     
        # return the warped image
        return warped
     
def doc_Scan(image):
        orig_height, orig_width = image.shape[:2]
        ratio = image.shape[0] / 500.0
     
        orig = image.copy()
        image = imutils.resize(image, height = 500)
        orig_height, orig_width = image.shape[:2]
        Original_Area = orig_height * orig_width
        
        # convert the image to grayscale, blur it, and find edges
        # in the image
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)
        edged = cv2.Canny(gray, 75, 200)
     
        cv2.imshow("Image", image)
        cv2.imshow("Edged", edged)
        cv2.waitKey(0)
        # show the original image and the edge detected image
     
        # find the contours in the edged image, keeping only the
        # largest ones, and initialize the screen contour
        contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key = cv2.contourArea, reverse = True)[:5]
        cv2.drawContours(image, contours, -1, (0,255,0), 3)
        cv2.imshow("Countor",image)
        cv2.waitKey(0)
        
        # loop over the contours
        for c in contours:
     
            # approximate the contour
            area = cv2.contourArea(c)
            if area < (Original_Area/4):
                print("Error Image Invalid")
                return("ERROR")
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
     
            # if our approximated contour has four points, then we
            # can assume that we have found our screen
            if len(approx) == 4:
                screenCnt = approx
                break
     
        # show the contour (outline) of the piece of paper
        cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
        cv2.imshow("Outline", image)
     
        warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
        # convert the warped image to grayscale, then threshold it
        # to give it that 'black and white' paper effect
        cv2.resize(warped, (640,403), interpolation = cv2.INTER_AREA)
        cv2.imwrite("credit_card_color.jpg", warped)
        warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        warped = warped.astype("uint8") * 255
        cv2.imshow("Extracted Credit Card", warped)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return warped

image = cv2.imread('debit.jpg')
cv2.imshow("normal",image)
cv2.waitKey(0)
image = doc_Scan(image)

dim = (640,403)

image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
#cv2.imwrite("credit_card_color.jpg", image)
#image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
region = [(40, 205), (640, 290)]
 
top_left_y = region[0][1]
bottom_right_y = region[1][1]
top_left_x = region[0][0]
bottom_right_x = region[1][0]
 
# Extracting the area were the credit numbers are located
roi = image[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
cv2.imshow("Region", roi)
cv2.imwrite("credit_card_extracted_digits.jpg", roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from keras.models import load_model

classifier = load_model("/home/saksham_nuclear_pc/IEEE Mega Project 3.0/creditcard.h5")

In [ ]:
import cv2
import numpy as np 
import random
import cv2
from scipy.ndimage import convolve
     
def DigitAugmentation(frame, dim = 32):
    frame = cv2.resize(frame, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
    random_num = np.random.randint(0,9)
     
    if (random_num % 2 == 0):
        frame = add_noise(frame)
    if(random_num % 3 == 0):
        frame = pixelate(frame)
    if(random_num % 2 == 0):
        frame = stretch(frame)
    frame = cv2.resize(frame, (dim, dim), interpolation = cv2.INTER_AREA)
     
    return frame 
     
def add_noise(image):
    prob = random.uniform(0.01, 0.05)
    rnd = np.random.rand(image.shape[0], image.shape[1])
    noisy = image.copy()
    noisy[rnd < prob] = 0
    noisy[rnd > 1 - prob] = 1
    return noisy
     
def pixelate(image):
    dim = np.random.randint(8,12)
    image = cv2.resize(image, (dim, dim), interpolation = cv2.INTER_AREA)
    image = cv2.resize(image, (16, 16), interpolation = cv2.INTER_AREA)
    return image
     
def stretch(image):
        ran = np.random.randint(0,3)*2
        if np.random.randint(0,2) == 0:
            frame = cv2.resize(image, (32, ran+32), interpolation = cv2.INTER_AREA)
            return frame[int(ran/2):int(ran+32)-int(ran/2), 0:32]
        else:
            frame = cv2.resize(image, (ran+32, 32), interpolation = cv2.INTER_AREA)
            return frame[0:32, int(ran/2):int(ran+32)-int(ran/2)]
        
def pre_process(image, inv = False):
        try:
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        except:
            gray_image = image
            pass
        
        if inv == False:
            _, th2 = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        else:
            _, th2 = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        resized = cv2.resize(th2, (32,32), interpolation = cv2.INTER_AREA)
        return resized

cc1 = cv2.imread('creditcard_digits2.jpg', 0)
_, th2 = cv2.threshold(cc1, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
#cv2.imshow("cc1", th2)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
     
region = [(0, 0), (35, 48)]
     
# Assigns values to each region for ease of interpretation
top_left_y = region[0][1]
bottom_right_y = region[1][1]
top_left_x = region[0][0]
bottom_right_x = region[1][0]
     
for i in range(0,1): #We only look at the first digit in testing out augmentation functions
    roi = cc1[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
    for j in range(0,10):
            roi2 = DigitAugmentation(roi)
            roi_otsu = pre_process(roi2, inv = False)
            #cv2.imshow("otsu", roi_otsu)
            #cv2.waitKey(0)
            
    cv2.destroyAllWindows()

In [ ]:
import cv2
import keras
def x_cord_contour(contours):
    #Returns the X cordinate for the contour centroid
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)
        return (int(M['m10']/M['m00']))
    else:
        pass
     
img = cv2.imread('credit_card_extracted_digits.jpg')
orig_img = cv2.imread('credit_card_color.jpg')
orig_img = cv2.resize(orig_img, dim, interpolation = cv2.INTER_AREA)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imshow("image", img)
cv2.waitKey(0)
     
# Blur image then find edges using Canny 
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#cv2.imshow("blurred", blurred)
#cv2.waitKey(0)
     
edged = cv2.Canny(blurred, 30, 150)
#cv2.imshow("edged", edged)
#cv2.waitKey(0)
     
# Find Contours
contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
     
#Sort out contours left to right by using their x cordinates
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:13] #Change this to 16 to get all digits
contours = sorted(contours, key = x_cord_contour, reverse = False)
     
# Create empty array to store entire number
full_number = []
     
    # loop over the contours
for c in contours:
        # compute the bounding box for the rectangle
    (x, y, w, h) = cv2.boundingRect(c)    
    if w >= 5 and h >= 25 and cv2.contourArea(c) < 1000:
            roi = blurred[y:y + h, x:x + w]
            #ret, roi = cv2.threshold(roi, 20, 255,cv2.THRESH_BINARY_INV)
            cv2.imshow("ROI1", roi)
            roi_otsu = pre_process(roi, True)
            cv2.imshow("ROI2", roi_otsu)
            roi_otsu = cv2.cvtColor(roi_otsu, cv2.COLOR_GRAY2RGB)
            roi_otsu = keras.preprocessing.image.img_to_array(roi_otsu)
            roi_otsu = roi_otsu * 1./255
            roi_otsu = np.expand_dims(roi_otsu, axis=0)
            image = np.vstack([roi_otsu])
            label = str(classifier.predict_classes(image, batch_size = 10))[1]
            print(label)
            (x, y, w, h) = (x+region[0][0]+40, y+region[0][1]+205, w, h)
            cv2.rectangle(orig_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(orig_img, label, (x , y + 90), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 2)
            cv2.imshow("image", orig_img)
            cv2.waitKey(0) 
            
cv2.destroyAllWindows()